## Capstone 1
To import the dataset I have simply copied the table from wikipedia into excel and save it as a csv file

In [1]:
#!conda install -c conda-forge geopy --yes 
#!conda install -c conda-forge folium=0.5.0 --yes

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
from jupyterthemes import jtplot
jtplot.style(theme='onedork')
#%matplotlib notebook
%matplotlib inline
import folium
from geopy.geocoders import Nominatim
import json
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
print("Libraries Imported")

Libraries Imported


## Read and check the dataset

In [3]:
df_toronto=pd.read_csv("C:\\Users\\...\\Documents\\Data_Analysis_Courses\\Data Science\\07_Capstone\\Toronto_Venues_800m.csv",index_col=0)
df_new_york=pd.read_csv("C:\\Users\\...\\Documents\\Data_Analysis_Courses\\Data Science\\07_Capstone\\New_York_Venues_800m.csv",index_col=0)
print("Toronto Dataset")
print(df_toronto.head())
print(df_toronto.shape)
print(df_toronto.columns)
print()
print("New York Dataset")
print(df_new_york.head())
print(df_new_york.shape)
print(df_new_york.columns)
print(np.unique(df_toronto['Neighborhood Latitude'].values))

Toronto Dataset
  Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0    Parkwoods                43.7588              -79.320197   
1    Parkwoods                43.7588              -79.320197   
2    Parkwoods                43.7588              -79.320197   
3    Parkwoods                43.7588              -79.320197   
4    Parkwoods                43.7588              -79.320197   

             Venue  Venue Latitude  Venue Longitude        Venue Category  
0  Allwyn's Bakery       43.759840       -79.324719  Caribbean Restaurant  
1             LCBO       43.757774       -79.314257          Liquor Store  
2      Tim Hortons       43.760668       -79.326368                  Café  
3       A&W Canada       43.760643       -79.326865  Fast Food Restaurant  
4        Dollarama       43.757317       -79.312578        Discount Store  
(9251, 7)
Index(['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude',
       'Venue', 'Venue Latitude', 'Venue Longitude', 

In [4]:
df_toronto_LatLong=pd.read_csv("C:\\Users\\...\\Documents\\Data_Analysis_Courses\\Data Science\\07_Capstone\\Toronto_Lat_Long.csv",index_col=0)
df_toronto_LatLong.columns=['Postcode','Borough','Neighborhood','Latitude','Longitude']
df_toronto_LatLong=df_toronto_LatLong[['Neighborhood','Latitude','Longitude']]
print("Toronto Dataset Latitude and Longitude of Neighborhoods")
print(df_toronto_LatLong.head())
print(df_toronto_LatLong.shape)
print(df_toronto_LatLong.columns)


Toronto Dataset Latitude and Longitude of Neighborhoods
       Neighborhood   Latitude  Longitude
0         Parkwoods  43.758800 -79.320197
1  Victoria Village  43.732658 -79.311189
2      Harbourfront  43.640080 -79.380150
3       Regent Park  43.660706 -79.360457
4  Lawrence Heights  43.722778 -79.450933
(210, 3)
Index(['Neighborhood', 'Latitude', 'Longitude'], dtype='object')


## Analysis of the venues

In [5]:
 np.unique(df_toronto['Venue Category'].values)

array(['Accessories Store', 'Afghan Restaurant', 'African Restaurant',
       'Airport', 'Airport Food Court', 'Airport Gate', 'Airport Lounge',
       'Airport Service', 'Airport Terminal', 'American Restaurant',
       'Amphitheater', 'Animal Shelter', 'Antique Shop', 'Aquarium',
       'Arcade', 'Art Gallery', 'Art Museum', 'Arts & Crafts Store',
       'Arts & Entertainment', 'Asian Restaurant', 'Athletics & Sports',
       'Auto Dealership', 'Automotive Shop', 'BBQ Joint', 'Baby Store',
       'Badminton Court', 'Bagel Shop', 'Bakery', 'Bank', 'Bar',
       'Baseball Field', 'Baseball Stadium', 'Basketball Court',
       'Basketball Stadium', 'Beach', 'Beach Bar', 'Beer Bar',
       'Beer Garden', 'Beer Store', 'Belgian Restaurant', 'Big Box Store',
       'Bike Shop', 'Bistro', 'Board Shop', 'Boat or Ferry', 'Bookstore',
       'Boutique', 'Bowling Alley', 'Brazilian Restaurant',
       'Breakfast Spot', 'Brewery', 'Bridal Shop', 'Bridge',
       'Bubble Tea Shop', 'Buffet', 'Bui

In [6]:
# one hot encoding
tor_venues_hot = pd.get_dummies(df_toronto[['Venue Category']], prefix="", prefix_sep="")
print(tor_venues_hot.shape)

# add neighborhood column back to dataframe
tor_venues_hot['Neighborhood'] = df_toronto['Neighborhood']
print(tor_venues_hot.shape)
# move neighborhood column to the first column
fixed_columns = [tor_venues_hot.columns[-1]] + list(tor_venues_hot.columns[:-1])
tor_venues_hot = tor_venues_hot[fixed_columns]

print(tor_venues_hot.shape)
tor_venues_hot.head()
#print(tor_venues_hot.loc[0])

(9251, 353)
(9251, 353)
(9251, 353)


,Zoo,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
tor_venues_group = tor_venues_hot.groupby('Neighborhood').mean().reset_index()
print(tor_venues_group.shape)
tor_venues_group.head()
#tor_venues_group['Coffee Shop'].head()
#tor_venues_group['Gym / Fitness Center'].head()

(206, 353)


,Neighborhood,Zoo,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,969 Eastern,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.0,0.000000,0.0,0.00,0.0,0.000000,0.0,0.0
1,Adelaide,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.02,0.000000,0.0,0.000000,0.0,0.01,0.0,0.000000,0.0,0.0
2,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.0,0.111111,0.0,0.00,0.0,0.000000,0.0,0.0
3,Agincourt North,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.034483,0.0,0.034483,0.0,0.00,0.0,0.034483,0.0,0.0
4,Albion Gardens,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.0,0.000000,0.0,0.00,0.0,0.000000,0.0,0.0


## Finding the top venues for each  Neighbourhood For Toronto

In [8]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [9]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tor_venues_group['Neighborhood']

for ind in np.arange(tor_venues_group.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_venues_group.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,969 Eastern,Brewery,Burrito Place,Pizza Place,Park,Sushi Restaurant
1,Adelaide,Coffee Shop,Restaurant,Hotel,Gastropub,Café
2,Agincourt,Chinese Restaurant,Asian Restaurant,Cantonese Restaurant,Vietnamese Restaurant,Hong Kong Restaurant
3,Agincourt North,Indian Restaurant,Park,Bakery,Ice Cream Shop,Taco Place
4,Albion Gardens,Grocery Store,Pharmacy,Electronics Store,Park,Hardware Store


## Clustering Analysis for Toronto

In [10]:
# set number of clusters
kclusters = 5

tor_venues_clustering = tor_venues_group.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_venues_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50] 

array([0, 0, 0, 0, 1, 1, 4, 0, 0, 4, 0, 0, 4, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 4, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

In [11]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
df_tor_cut_merge = df_toronto_LatLong.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

df_tor_cut_merge.head() # check the last columns!

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Parkwoods,43.758800,-79.320197,1,Coffee Shop,Supermarket,Discount Store,Pet Store,Pharmacy
1,Victoria Village,43.732658,-79.311189,0,Intersection,Middle Eastern Restaurant,Asian Restaurant,Chinese Restaurant,French Restaurant
2,Harbourfront,43.640080,-79.380150,0,Coffee Shop,Hotel,Aquarium,Café,Scenic Lookout
3,Regent Park,43.660706,-79.360457,0,Coffee Shop,Restaurant,Park,Pub,Breakfast Spot
4,Lawrence Heights,43.722778,-79.450933,0,Clothing Store,Coffee Shop,Fast Food Restaurant,Restaurant,Dessert Shop


In [12]:
latitude=43.6532
longitude=-79.3832
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_tor_cut_merge['Latitude'], df_tor_cut_merge['Longitude'], df_tor_cut_merge['Neighborhood'], df_tor_cut_merge['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Finding the top venues for each  Neighbourhood For New York

In [13]:
# one hot encoding
NY_venues_hot = pd.get_dummies(df_new_york[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
NY_venues_hot['Neighborhood'] = df_new_york['Neighborhood']
# move neighborhood column to the first column
fixed_columns = [NY_venues_hot.columns[-1]] + list(NY_venues_hot.columns[:-1])
NY_venues_hot = NY_venues_hot[fixed_columns]

print(tor_venues_hot.shape)
NY_venues_hot.head()
#print(tor_venues_hot.loc[0])

(9251, 353)


,Zoo Exhibit,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,...,Waterfront,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
NY_venues_group = NY_venues_hot.groupby('Neighborhood').mean().reset_index()
print(NY_venues_group.shape)
NY_venues_group.head()

(302, 462)


,Neighborhood,Zoo Exhibit,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Waterfront,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Allerton,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Annadale,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Arden Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Arlington,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.076923,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Arrochar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted_NY = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_NY['Neighborhood'] = NY_venues_group['Neighborhood']

for ind in np.arange(NY_venues_group.shape[0]):
    neighborhoods_venues_sorted_NY.iloc[ind, 1:] = return_most_common_venues(NY_venues_group.iloc[ind, :], num_top_venues)

print(neighborhoods_venues_sorted_NY.shape)
neighborhoods_venues_sorted_NY.head()

(302, 6)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Allerton,Donut Shop,Pizza Place,Sandwich Place,Pharmacy,Bus Station
1,Annadale,Pizza Place,Restaurant,Deli / Bodega,Train Station,Bakery
2,Arden Heights,Mexican Restaurant,Italian Restaurant,Pizza Place,Shopping Mall,Bank
3,Arlington,Bus Stop,Deli / Bodega,Home Service,American Restaurant,Coffee Shop
4,Arrochar,Deli / Bodega,Bus Stop,Pizza Place,Italian Restaurant,Liquor Store


## Clustering Analysis for New York

In [16]:
# set number of clusters
kclusters = 5

NY_venues_clustering = NY_venues_group.drop('Neighborhood', 1)

# run k-means clustering
NY_kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(NY_venues_clustering)

# check cluster labels generated for each row in the dataframe
NY_kmeans.labels_[0:50] 

array([2, 2, 4, 3, 3, 2, 4, 2, 4, 4, 4, 4, 4, 2, 4, 0, 2, 4, 2, 2, 1, 2,
       4, 2, 1, 4, 4, 4, 2, 4, 2, 4, 2, 2, 2, 4, 3, 2, 2, 4, 1, 2, 2, 4,
       4, 2, 2, 4, 4, 4])

In [17]:
# add clustering labels
neighborhoods_venues_sorted_NY.insert(0, 'Cluster Labels', NY_kmeans.labels_)
#neighborhoods_venues_sorted_NY.head()


# merge new york grouped with toronto_data to add latitude/longitude for each neighborhood
new_york_grouped=df_new_york[['Neighborhood','Neighborhood Latitude','Neighborhood Longitude']].groupby('Neighborhood',as_index=False).mean()
new_york_grouped.columns=['Neighborhood','Latitude','Longitude']

df_NY_cut_merge = new_york_grouped.join(neighborhoods_venues_sorted_NY.set_index('Neighborhood'), on='Neighborhood',how="inner")

print(df_NY_cut_merge.shape)
print(df_NY_cut_merge.dropna().shape)
df_NY_cut_merge.head() # check the last columns!

(302, 9)
(302, 9)


,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Allerton,40.865788,-73.859319,2,Donut Shop,Pizza Place,Sandwich Place,Pharmacy,Bus Station
1,Annadale,40.538114,-74.178549,2,Pizza Place,Restaurant,Deli / Bodega,Train Station,Bakery
2,Arden Heights,40.549286,-74.185887,4,Mexican Restaurant,Italian Restaurant,Pizza Place,Shopping Mall,Bank
3,Arlington,40.635325,-74.165104,3,Bus Stop,Deli / Bodega,Home Service,American Restaurant,Coffee Shop
4,Arrochar,40.596313,-74.067124,3,Deli / Bodega,Bus Stop,Pizza Place,Italian Restaurant,Liquor Store


In [18]:
latitude=40.7128
longitude=-74.0060
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_NY_cut_merge['Latitude'], df_NY_cut_merge['Longitude'], df_NY_cut_merge['Neighborhood'], df_NY_cut_merge['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Analysis of various groups

In [19]:
df_tor_cut_merge.loc[df_tor_cut_merge['Cluster Labels'] == 0, df_tor_cut_merge.columns[[0] + list(range(3, df_tor_cut_merge.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,Victoria Village,0,Intersection,Middle Eastern Restaurant,Asian Restaurant,Chinese Restaurant,French Restaurant
2,Harbourfront,0,Coffee Shop,Hotel,Aquarium,Café,Scenic Lookout
3,Regent Park,0,Coffee Shop,Restaurant,Park,Pub,Breakfast Spot
4,Lawrence Heights,0,Clothing Store,Coffee Shop,Fast Food Restaurant,Restaurant,Dessert Shop
5,Lawrence Manor,0,Park,Bagel Shop,Doctor's Office,Sandwich Place,Bank
6,Queen's Park,0,Coffee Shop,Café,Ice Cream Shop,Bubble Tea Shop,Burger Joint
7,Islington Avenue,0,Coffee Shop,Pub,Restaurant,Gym,Gym / Fitness Center
10,Don Mills North,0,Women's Store,Pizza Place,Restaurant,American Restaurant,Coffee Shop
13,Ryerson,0,Coffee Shop,Cosmetics Shop,Clothing Store,Tea Room,Restaurant
14,Garden District,0,Coffee Shop,Italian Restaurant,Cosmetics Shop,Clothing Store,Gastropub


In [20]:
df_NY_cut_merge.loc[df_NY_cut_merge['Cluster Labels'] == 2, df_NY_cut_merge.columns[[0] + list(range(3, df_NY_cut_merge.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Allerton,2,Donut Shop,Pizza Place,Sandwich Place,Pharmacy,Bus Station
1,Annadale,2,Pizza Place,Restaurant,Deli / Bodega,Train Station,Bakery
5,Arverne,2,Surf Spot,Beach,Sandwich Place,Deli / Bodega,Bus Stop
7,Astoria Heights,2,Italian Restaurant,Pizza Place,Bus Station,Airport Terminal,Bakery
13,Baychester,2,Clothing Store,Pharmacy,Department Store,Donut Shop,Bus Station
16,Bedford Park,2,Pizza Place,Deli / Bodega,Supermarket,Diner,Chinese Restaurant
18,Beechhurst,2,Donut Shop,Pizza Place,Italian Restaurant,Bus Station,Chinese Restaurant
19,Bellaire,2,Pizza Place,Bus Station,Chinese Restaurant,Intersection,Fast Food Restaurant
21,Bellerose,2,Deli / Bodega,Pizza Place,Mobile Phone Shop,Italian Restaurant,Bus Station
23,Bensonhurst,2,Bakery,Japanese Restaurant,Pizza Place,Donut Shop,Chinese Restaurant


In [21]:
df_tor_cut_merge.loc[df_tor_cut_merge['Cluster Labels'] == 1, df_tor_cut_merge.columns[[1] + list(range(5, df_tor_cut_merge.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,43.758800,Supermarket,Discount Store,Pet Store,Pharmacy
9,43.809196,Pizza Place,Pharmacy,Fast Food Restaurant,Sandwich Place
11,43.712078,Pharmacy,Fast Food Restaurant,Coffee Shop,Grocery Store
12,43.706298,Intersection,Soccer Stadium,Brewery,Skating Rink
16,43.633637,Supermarket,Sandwich Place,Coffee Shop,Clothing Store
17,43.645338,Pub,Fast Food Restaurant,Sushi Restaurant,Grocery Store
22,43.780271,Sandwich Place,Pet Store,Liquor Store,Train Station
23,43.775504,Sandwich Place,Pet Store,Liquor Store,Train Station
29,43.635317,Sandwich Place,Coffee Shop,Bookstore,Gym / Fitness Center
32,43.639373,Pharmacy,Baseball Field,Liquor Store,Beer Store


In [22]:
df_tor_cut_merge.loc[df_tor_cut_merge['Cluster Labels'] == 2, df_tor_cut_merge.columns[[1] + list(range(5, df_tor_cut_merge.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
204,43.641597,Hostel,Dive Bar,Doctor's Office,Dog Run


In [23]:
df_tor_cut_merge.loc[df_tor_cut_merge['Cluster Labels'] == 3, df_tor_cut_merge.columns[[1] + list(range(5, df_tor_cut_merge.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
8,43.80493,Fast Food Restaurant,Bus Station,Park,Yoga Studio
187,43.80493,Fast Food Restaurant,Bus Station,Park,Yoga Studio


In [24]:
df_tor_cut_merge.loc[df_tor_cut_merge['Cluster Labels'] == 4, df_tor_cut_merge.columns[[1] + list(range(5, df_tor_cut_merge.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
20,43.663199,Convenience Store,Electronics Store,Park,Ethiopian Restaurant
21,43.790117,Breakfast Spot,Bus Station,Baseball Field,Italian Restaurant
26,43.699930,Park,Intersection,Bus Stop,Pharmacy
30,43.662273,Coffee Shop,Convenience Store,Eastern European Restaurant,Park
31,43.631238,Flower Shop,Golf Course,Bank,Baseball Field
45,43.763893,Park,Playground,Bagel Shop,Convenience Store
56,43.769509,Park,Intersection,Tennis Court,Electronics Store
58,43.754135,Park,Moving Target,Electronics Store,Dive Bar
92,43.760078,Sports Bar,Mexican Restaurant,Coffee Shop,Bakery
108,43.702112,Coffee Shop,Gym Pool,General Entertainment,American Restaurant
